### Gabor filter for HPMF
In this file there is gabor filter used for HPMF file.
Source: https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_gabor.html

And writing help from ChatGPT.
Rasterio making sure your GEOTiff does not loose the projection. 

About 140 min, 64 Gt RAM and 16 images. From the wide area, bigger than the study area. In the end, images were clipped to study area zones (21 zones).

Differnece within sigma 0.5 and sigma 1 was small. In the end sigma 1 images were used. 

In [1]:
import numpy as np
from scipy import ndimage as ndi
from skimage import io, img_as_float
from skimage.filters import gabor_kernel, gaussian
import os
import rasterio
from rasterio.transform import from_origin  # Importing from_origin for spatial transform

In [2]:
# Paths
input_path = "E:/Gradu/HighPassMedianFilter/hpmf_re0_5_cl.tif"
output_dir = "E:/Gradu/Gabor/Filtered_Images/"
os.makedirs(output_dir, exist_ok=True)

# Load the High Pass Median Filter image
image = img_as_float(io.imread(input_path))

# Apply Gaussian filter to reduce noise
image_gaussian = gaussian(image, sigma=0.5)  # Adjust sigma as needed for noise reduction, first here was sigma=1, then sigma=0.5, in the end just sigma=1 was used in here.

# Define the projection and transform details (assuming top-left coordinates and pixel size are known)
pixel_size = 0.5  # Each pixel represents 0.5m x 0.5m
top_left_x, top_left_y = 368027, 6875970  # Adjust based on your image’s actual coordinates
transform = from_origin(top_left_x, top_left_y, pixel_size, pixel_size)
crs = "EPSG:3067"  # ETRS89 / TM35FIN projection

# Prepare Gabor filter bank kernels
kernels = []
for theta in range(4):  # Four orientations
    theta = theta / 4.0 * np.pi
    for sigma in (1, 3):  # Different scales
        for frequency in (0.05, 0.25):  # Frequencies for edge/line detection
            kernel = np.real(gabor_kernel(frequency, theta=theta, sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

# Apply each Gabor filter to the Gaussian-filtered image and save the output with EPSG projection
for i, kernel in enumerate(kernels):
    # Convolve the image with the Gabor kernel
    filtered_image = ndi.convolve(image_gaussian, kernel, mode='wrap')

    # Normalize filtered image for saving as uint8 (if applicable)
    filtered_image_normalized = (filtered_image - np.min(filtered_image)) / (np.max(filtered_image) - np.min(filtered_image))  # Normalize to 0-1
    filtered_image_uint8 = (filtered_image_normalized * 255).astype(np.uint8)  # Convert to uint8

    # Save the filtered image with rasterio, including transform and CRS
    output_path = os.path.join(output_dir, f"HPMFgabor_filtered_sigma05_{i}.tif")
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=filtered_image.shape[0],
        width=filtered_image.shape[1],
        count=1,
        dtype='uint8',  # Save as uint8 to reduce size
        crs=crs,
        transform=transform,
        compress='lzw'  # Apply compression
    ) as dst:
        dst.write(filtered_image_uint8, 1)

print("Gabor filtered images with EPSG:3067 saved to:", output_dir)

Gabor filtered images with EPSG:3067 saved to: E:/Gradu/Gabor/Filtered_Images/
